# Nettoyage des données

Import des modules

In [2]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt

# Display plots inline and change default figure size
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0)

Import des données (prends plusieurs minutes)

In [3]:
tic = time.time()
data = pd.read_csv('data/data_18_19_orig.csv')
tac = time.time()

print("Temps de chargement: " + str(tac-tic) + " s")

/Users/gauthier/workspace/py_env/3_ML_DL/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Temps de chargement: 21.795127153396606 s


In [4]:
data.head()

,CAB,Date de passage,Date de sortie,Emetteur,Code Regate Emetteur,Destination,Code Regate Destinataire,Produit,Traitement,Code traitement,Contenant,Injecteur,Sortie,Rejet,CAB fonctionnel,CBN
0,2013373309901180831677726030,2018-08-01 06:00:48,2018-08-01 06:01:34,BORDEAUX PIC,337330,TRI MANUEL 2,990118,LETTRE,TRI MANUEL TG2,26.0,ké7,11.0,37,NON,-01------99011808------26-30,non trouvé
1,2013373303323100831677937017,2018-08-01 06:00:51,2018-08-01 06:01:32,BORDEAUX PIC,337330,BORDEAUX PDC1,332310,LETTRE,TRI CODE POSTAL,37.0,ké7,11.0,LV17,NON,-01------33231008------37-17,Q 03
2,2013373309901180821853326030,2018-08-01 06:03:31,2018-08-01 06:04:17,BORDEAUX PIC,337330,TRI MANUEL 2,990118,LETTRE,TRI MANUEL TG2,26.0,ké7,11.0,37,NON,-01------99011808------26-30,non trouvé
3,2223373303373300349894290000,2018-08-01 06:10:55,2018-08-01 06:11:02,BORDEAUX PIC,337330,BORDEAUX PIC,337330,ECOPLI,TRI LECTEUR AUTO TOUS FLUX,90.0,bak,NaN,MTIGF02,NON,-22------33733003------90-00,non trouvé
4,2223373303373300349894390000,2018-08-01 06:10:57,2018-08-01 06:11:03,BORDEAUX PIC,337330,BORDEAUX PIC,337330,ECOPLI,TRI LECTEUR AUTO TOUS FLUX,90.0,bak,NaN,MTIGF02,NON,-22------33733003------90-00,non trouvé


Dates au format Timestamp

In [5]:
data['Date de passage'] = data['Date de passage'].apply(lambda x: pd.Timestamp(x))
data['Date de sortie'] = data['Date de sortie'].apply(lambda x: pd.Timestamp(x))

## Nettoyage des codes CBN

Affichage des codes batailles navales. Les codes commençant par un "O" sont rentrés avec un "0" (zéro).

In [6]:
data['CBN'].unique()

array(['non trouvé', 'Q 03', '0 05', 'ligne vide', '0 09', '0 06', '0 03',
       '0 04', '0 02', 'G 01', 'C 01', 'CAB nul', 'N 03', 'L 01', '0 11',
       '0 07', '0 10', 'D 02', 'F 01', '0 08', 'N 02', 'T 02', 'N 11',
       'Y', 'A 04/ A 06', 'W 04', 'D 01', 'A 08', 'BRIN A', 'B 03', 'Z',
       'S 05', 'V 09', 'BRIN B', 'H 06', 'C 02', 'X 01/X 02', 'M 10',
       'H 09', 'Q 01', 'I 03/I 04/I 05/I 06/I 07/I 08', 'V 08', 'W 01',
       'M 12', 'M 07', 'F 04', 'G 05', 'E 12', 'U 09', 'M 05', 'D 10',
       'E 11', 'D 07', 'E 06', 'K 04', 'W 05', 'W 03', 'A 07', 'K 05',
       'K 02', 'N 01', 'G 04', 'E 10', 'W 02', 'K 06', 'J 03',
       'C 04/C 05', 'V 02', 'BRIN C', 'B 05/B 06', 'S 09', 'H 11', 'D 04',
       'T 06/T 07', 'F 08', 'W 06', 'V 03', 'A 10', 'J 02', 'J 01',
       'L 10', 'S 01', 'D 09', 'U 11', 'V 04', 'V 01', 'I 02', 'D 06',
       'S 04', 'V 06', 'B 08', 'M 03', 'I 11/I 12', 'I 10', 'B 09',
       'I 11', 'S 02', 'H 08', 'S 03', 'I 12', 'E 03/E 04', '0 12',
       'V 

Analyse des codes suivants:
    - 'non trouvé'
    - 'ligne vide'

In [7]:
nb_nt = data[data['CBN'] == 'non trouvé'].shape[0]
nb_vide = data[data['CBN'] == 'ligne vide'].shape[0]

size = data.shape[0]

print(str(nb_nt) + " correspondent à un CBN 'non trouvé' soit " +str(int(nb_nt/size*100))+ " % des données.")
print(str(nb_vide) + " correspondent à un CBN 'ligne vide' soit " +str(int(nb_vide/size*100))+ " % des données.")
print('\n')

print("Conclusion : "+str(int((nb_nt+nb_vide)/size*100))+" % des données ne sont pas utilisables")

1513738 correspondent à un CBN 'non trouvé' soit 19 % des données.
462201 correspondent à un CBN 'ligne vide' soit 6 % des données.


Conclusion : 25 % des données ne sont pas utilisables


Supression des lignes correspondantes à ces CBN:

In [7]:
data = data[(data['CBN'] != 'ligne vide') & (data['CBN'] != 'non trouvé')]

Les colonnes suivantes sont supprimées car inutiles à notre analyse:
    - "CAB"
    - "Emetteur"
    - "Code Regate Emetteur"
    - "Destination"
    - "Code Regate Destinataire"
    - "Produit"
    - "Traitement"
    - "Code traitement"
    - "Injecteur"
    - "Sortie"

In [10]:
data.drop(columns=['CAB', 'Emetteur', 'Code Regate Emetteur', 'Destination', 'Code Regate Destinataire', \
                  'Produit', 'Traitement', 'Code traitement', 'Injecteur', 'Sortie'], inplace=True)

In [19]:
data.reset_index(inplace=True)

In [21]:
data.head()

,index,Date de passage,Date de sortie,Contenant,Rejet,CAB fonctionnel,CBN
0,1,2018-08-01 06:00:51,2018-08-01 06:01:32,ké7,NON,-01------33231008------37-17,Q 03
1,21,2018-08-01 06:18:43,2018-08-01 06:18:43,bak,NON,-01------99011903------51-00,0 05
2,22,2018-08-01 06:18:46,2018-08-01 06:18:46,bak,NON,-01------99011903------51-00,0 05
3,23,2018-08-01 06:18:51,2018-08-01 06:18:51,bak,NON,-01------99011903------51-00,0 05
4,24,2018-08-01 06:18:54,2018-08-01 06:18:54,bak,NON,-01------99011903------51-00,0 05


In [ ]:
data.to_csv('data/data_18_19_clean.csv', index=False)